# AI-agenten bouwen met blijvend geheugen met behulp van Cognee

Deze notebook laat zien hoe je intelligente AI-agenten kunt bouwen met geavanceerde geheugenmogelijkheden met behulp van [**cognee**](https://www.cognee.ai/) - een open source AI-geheugen dat kennisgrafieken, semantisch zoeken en sessiebeheer combineert om contextbewuste AI-systemen te creëren.

## 🎯 Leerdoelen

Aan het einde van deze tutorial begrijp je hoe je:
- **Kennisgrafieken ondersteund door embeddings bouwt**: Ongeordende tekst omzet in gestructureerde, doorzoekbare kennis
- **Sessiegeheugen implementeert**: Meerdere gespreksrondes creëert met automatische contextbehoud
- **Gesprekken bewaart**: Belangrijke interacties optioneel opslaat in langetermijngeheugen voor toekomstig gebruik
- **Vraagt met natuurlijke taal**: Historische context gebruikt en toepast in nieuwe gesprekken
- **Geheugen visualiseert**: De relaties in de kennisgrafiek van je agent verkent


## 🏗️ Wat Je Gaat Bouwen

In deze tutorial maken we een **Coding Assistant** met blijvend geheugen die:

### 1. **Kennisbank Constructie**
   - Ontwikkelaarsprofielen en expertise-informatie verwerkt
   - Principes en best practices van Python-programmeren verwerkt
   - Historische gesprekken tussen ontwikkelaars en AI-assistenten opslaat

### 2. **Sessie-bewuste Gesprekken**
   - Context behoudt over meerdere vragen binnen dezelfde sessie
   - Automatisch elk vraag/antwoord-paar cachet voor efficiënte terugwinning
   - Coherente, contextuele antwoorden biedt op basis van gespreksgeschiedenis

### 3. **Langetermijngeheugen**
   - Belangrijke gesprekken opslaat in een langetermijngeheugen
   - Relevante herinneringen uit de kennisbank en eerdere sessies ophaalt om nieuwe interacties te informeren
   - Een groeiende kennisbank opbouwt die in de loop van de tijd verbetert

### 4. **Intelligente Geheugenopvraging**
   - Grafiek-bewuste semantische zoekopdrachten gebruikt om relevante informatie te vinden in alle opgeslagen kennis
   - Zoekopdrachten filtert op datasubgroepen (ontwikkelaarsinfo vs. principes)
   - Meerdere gegevensbronnen combineert om uitgebreide antwoorden te bieden


## 📋 Vereisten & Installatie

### Systeemvereisten

Voordat je begint, zorg ervoor dat je het volgende hebt:

1. **Python-omgeving**
   - Python 3.9 of hoger
   - Virtuele omgeving (aanbevolen)
   
2. **Redis Cache** (Vereist voor sessiebeheer)
   - Lokale Redis: `docker run -d -p 6379:6379 redis`
   - Of gebruik een beheerde Redis-service
   
3. **LLM API-toegang**
   - OpenAI API-sleutel of andere providers (zie [documentatie](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Databaseconfiguratie**
   - Standaard is geen configuratie nodig. Cognee gebruikt bestandsgebaseerde databases (LanceDB en Kuzu)
   - Optioneel kun je Azure AI Search instellen als vectoropslag (zie [documentatie](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Omgevingsconfiguratie

Maak een `.env`-bestand in je projectmap met de volgende variabelen:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Begrijpen van Cognee's Geheugenarchitectuur

### Hoe Cognee Werkt

Cognee biedt een geavanceerd geheugensysteem dat verder gaat dan eenvoudige key-value opslag:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Belangrijke Componenten:

1. **Kennisgrafiek**: Slaat entiteiten, relaties en semantische verbindingen op
2. **Vector-Embeddings**: Maakt semantisch zoeken mogelijk in alle opgeslagen informatie
3. **Sessiecache**: Behoudt de gesprekscontext binnen en tussen sessies
4. **NodeSets**: Organiseren gegevens in logische categorieën voor gerichte opvraging

### Geheugentypen in Deze Tutorial:

- **Persistent Geheugen**: Langdurige opslag in de kennisgrafiek
- **Sessiegeheugen**: Tijdelijke gesprekscontext in Redis-cache
- **Semantisch Geheugen**: Vector-gebaseerd zoeken naar overeenkomsten in alle data


## 📦 Vereiste pakketten installeren

Installeer Cognee met Redis-ondersteuning voor sessiebeheer:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Initialiseer omgeving en laad bibliotheken

Zorg ervoor dat:
1. Redis actief is (bijvoorbeeld via Docker: `docker run -d -p 6379:6379 redis`)
2. Omgevingsvariabelen zijn ingesteld voordat cachemodules worden geïmporteerd
3. Indien nodig, de kernel opnieuw opstarten en de cellen in volgorde uitvoeren

De volgende cel zal:
1. Omgevingsvariabelen laden uit `.env`
2. Cognee configureren met je LLM-instellingen
3. Caching inschakelen voor sessiebeheer
4. Controleren of alle componenten correct zijn verbonden


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Opslagmappen configureren

Cognee gebruikt twee aparte mappen voor zijn operaties:
- **Data Root**: Slaat geïmporteerde documenten en verwerkte data op
- **System Root**: Bevat de kennisgrafiekdatabase en systeemmetadata

We zullen voor deze tutorial geïsoleerde mappen aanmaken als volgt:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Geheugenstatus resetten

Voordat we beginnen met het bouwen van ons geheugensysteem, laten we ervoor zorgen dat we met een schone lei beginnen.

> 💡 **Tip**: Je kunt deze stap overslaan als je bestaande herinneringen van je eerdere sessies wilt behouden wanneer je dit notebook later gebruikt.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Deel 1: Het opbouwen van de kennisbank

### Gegevensbronnen voor onze ontwikkelaarassistent

We zullen drie soorten gegevens verzamelen om een uitgebreide kennisbank te creëren:

1. **Ontwikkelaarsprofiel**: Persoonlijke expertise en technische achtergrond
2. **Python Best Practices**: The Zen of Python met praktische richtlijnen
3. **Historische gesprekken**: Eerdere Q&A-sessies tussen ontwikkelaars en AI-assistenten

Deze diverse gegevens stellen onze agent in staat om:
- De technische context van de gebruiker te begrijpen
- Best practices toe te passen in aanbevelingen
- Te leren van eerdere succesvolle interacties


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Verwerk gegevens tot een kennisgrafiek

Nu gaan we onze ruwe tekst omzetten in een gestructureerd geheugen. Dit proces:

1. **Voegt gegevens toe aan NodeSets**: Organiseert informatie in logische categorieën
   - `developer_data`: Ontwikkelaarsprofiel en gesprekken
   - `principles_data`: Python best practices en richtlijnen

2. **Voert Cognify Pipeline uit**: Extraheert entiteiten, relaties en creëert embeddings
   - Identificeert sleutelconcepten
   - Maakt semantische verbindingen tussen gerelateerde informatie
   - Genereert vector-embeddings

Dit kan even duren terwijl de LLM de tekst verwerkt en de grafiekstructuur opbouwt:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualiseer de Kennisgrafiek

Laten we de structuur van onze kennisgrafiek verkennen. De visualisatie toont:
- **Knopen**: Entiteiten die uit de tekst zijn gehaald (concepten, technologieën, personen)
- **Randen**: Relaties en verbindingen tussen entiteiten
- **Clusters**: Gerelateerde concepten gegroepeerd op semantische gelijkenis

Open het gegenereerde HTML-bestand in je browser om de grafiek interactief te verkennen:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Verrijk Geheugen met Memify

De functie `memify()` analyseert de kennisgrafiek en genereert intelligente regels over de data. Dit proces:
- Herkent patronen en best practices
- Creëert uitvoerbare richtlijnen op basis van de inhoud
- Legt verbanden tussen verschillende kennisgebieden

Deze regels helpen de agent om beter geïnformeerde beslissingen te nemen bij het beantwoorden van vragen. Het vastleggen van een tweede visualisatie helpt je te vergelijken hoe de grafiek dichter wordt zodra deze verrijkt is.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Deel 2: Intelligente Geheugenopvraging

### Demonstratie 1: Kennisintegratie over Documenten heen

Nu onze kennisgrafiek is opgebouwd, laten we testen hoe Cognee informatie uit meerdere bronnen combineert om complexe vragen te beantwoorden.

De eerste query demonstreert:
- **Semantisch begrip**: Het vinden van relevante concepten, zelfs als ze niet expliciet worden genoemd
- **Kruisverwijzing**: Het combineren van ontwikkelaarsprofielen met Python-principes
- **Contextueel redeneren**: Het toepassen van best practices op specifieke implementaties

### Demonstratie 2: Gefilterd Zoeken met NodeSets

De tweede query laat zien hoe je specifieke subsets van de kennisgrafiek kunt targeten:
- Gebruikt de `node_name` parameter om alleen binnen `principles_data` te zoeken
- Biedt gerichte antwoorden uit een specifieke kennisdomein
- Handig wanneer je domeinspecifieke informatie nodig hebt


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Deel 3: Sessiebeheer Instellen

### Gespreksgeheugen Inschakelen

Sessiebeheer is essentieel om context te behouden over meerdere interacties. Hier zullen we:

1. **Gebruikerscontext Initialiseren**: Een gebruikersprofiel aanmaken of ophalen voor sessietracking
2. **Cache Engine Configureren**: Verbinden met Redis om gespreksgeschiedenis op te slaan
3. **Sessies Variabelen Inschakelen**: Contextvariabelen instellen die blijven bestaan over meerdere vragen

> ⚠️ **Belangrijk**: Dit vereist dat Redis actief is en `CACHING=true` in je omgeving is ingesteld


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Hulpfunctie: Bekijk sessiegeschiedenis

Deze hulpfunctie stelt ons in staat om de gespreksgeschiedenis opgeslagen in Redis te bekijken. Het is handig voor:
- Het debuggen van sessiebeheer
- Controleren of gesprekken worden opgeslagen
- Begrijpen welke context beschikbaar is voor de agent


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sessie 1: Async Support Lab — Eerste Vraag

Start de sessie `async-support-lab` door te vragen naar telemetrie-vriendelijke asyncio-patronen voor een grootschalige webscraper. De grafiek is al bekend met asyncio, aiohttp en monitoringpraktijken, dus het antwoord moet eerdere gesprekken weerspiegelen terwijl het wordt afgestemd op de nieuwe vraag.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspecteer het geheugen van Sessie 1 na de eerste uitwisseling

Het uitvoeren van `show_history(session_1)` direct na de eerste vraag bevestigt dat Cognee zowel de prompt als de voltooiing in Redis heeft opgeslagen. Je zou één invoer moeten zien met de richtlijnen voor gelijktijdigheid.


In [ ]:
await show_history(session_1)

## Sessie 1: Vervolg op Datamodellen

Vervolgens vragen we: "Wanneer moet ik kiezen voor dataclasses in plaats van Pydantic?" met gebruik van dezelfde sessie-id. Cognee zou de Python-principes en eerdere FastAPI-gesprekken moeten combineren om genuanceerd advies te geven—waarbij wordt aangetoond dat context binnen een benoemde sessie wordt meegenomen.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Bevestig dat de geschiedenis van sessie 1 beide beurten bevat

Een andere `show_history(session_1)` oproep zou twee vraag-en-antwoord items moeten tonen. Dit komt overeen met de "geheugenherhaling" stap van het Mem0-lab en bewijst dat extra beurten hetzelfde transcript uitbreiden.


In [ ]:
await show_history(session_1)

## Sessie 2: Ontwerpreview-thread — Nieuwe sessie

Om isolatie tussen threads te laten zien, starten we `design-review-session` en vragen we om logboekrichtlijnen voor incidentreviews. Hoewel de onderliggende kennisbasis hetzelfde is, zorgt de nieuwe sessie-id ervoor dat transcripties gescheiden blijven.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Beoordelingssessie 2 Geschiedenis

`show_history(session_2)` zou alleen het ontwerpbeoordelingsprompt/antwoordpaar moeten weergeven. Vergelijk het met Sessie 1 om te benadrukken hoe Cognee onafhankelijke transcripties behoudt terwijl het de gedeelde kennisgrafiek hergebruikt.


In [ ]:
await show_history(session_2)

## Samenvatting

Gefeliciteerd! Je hebt je code-assistent zojuist een echte langetermijngeheugenlaag gegeven, aangedreven door Cognee.

In deze tutorial heb je ruwe ontwikkelaarsinhoud (code, documentatie, chats) omgezet in een grafiek + vectorgeheugen waarmee je agent kan zoeken, redeneren en continu verbeteren.

Wat je hebt geleerd

1. **Van ruwe tekst naar AI-geheugen**: Hoe Cognee ongestructureerde data verwerkt en omzet in intelligent, doorzoekbaar geheugen met behulp van een gecombineerde vector- en kennisgrafiekarchitectuur.

2. **Grafiekverrijking met memify**: Hoe je verder kunt gaan dan basisgrafiekcreatie en memify kunt gebruiken om afgeleide feiten en rijkere relaties toe te voegen aan je bestaande grafiek.

3. **Meerdere zoekstrategieën**: Hoe je geheugen kunt doorzoeken met verschillende zoektypen (grafiekbewuste Q&A, RAG-stijl aanvulling, inzichten, ruwe fragmenten, codezoekopdrachten, enz.) afhankelijk van wat je agent nodig heeft.

4. **Visuele verkenning**: Hoe je kunt inspecteren en debuggen wat Cognee heeft gebouwd met behulp van grafiekvisualisaties en de Cognee UI, zodat je daadwerkelijk kunt zien hoe kennis is gestructureerd.

5. **Sessie-bewust geheugen**: Hoe je sessiecontext kunt combineren met persistent semantisch geheugen, zodat agents zich sessie-overstijgend kunnen herinneren zonder informatie tussen gebruikers te lekken.


## Belangrijkste punten
1. Geheugen als een Kennisgrafiek ondersteund door Embeddings

    - **Gestructureerd begrip**: Cognee combineert een vectoropslag en een grafiekopslag, zodat je gegevens zowel door betekenis doorzoekbaar zijn als verbonden door relaties. Standaard gebruikt Cognee bestand-gebaseerde databases (LanceDB voor vector-, Kuzu voor grafiekdatabase).

    - **Relatiebewuste opvraging**: Antwoorden kunnen niet alleen gebaseerd zijn op "vergelijkbare tekst", maar ook op hoe entiteiten met elkaar in verband staan.

    - **Levend geheugen**: De geheugenslaag evolueert, groeit en blijft doorzoekbaar als één verbonden grafiek.

2. Zoek- en redeneermodi
    - **Hybride opvraging**: zoeken combineert vectorovereenkomst, grafiekstructuur en LLM-redenering, van ruwe chunk-opzoeking tot grafiekbewuste vraagbeantwoording.

    - **Pas de modus aan de taak aan**: Gebruik completion-stijl modi wanneer je natuurlijke taalantwoorden wilt, en chunk-/samenvatting-/grafiekmodi wanneer je agent ruwe context nodig heeft of zijn eigen redenering moet sturen.

3. Gepersonaliseerde, sessiebewuste agents
    - **Sessiecontext + langetermijngeheugen**: Cognee houdt kortetermijn "thread"-context gescheiden van langdurig, gebruikers- of organisatieniveau geheugen.

## Toepassingen in de praktijk

1. **Verticale AI-agents**

    Gebruik het patroon uit dit notebook om domeinslimme copilots te ondersteunen die bovenop Cognee draaien als hun kern voor opvraging en redenering:

- **Ontwikkelaars-copilots**: Code review, incidentanalyse en architectuurassistenten die code, API's, ontwerpdocumenten en tickets doorzoeken als één geheugenkaart.

- **Klantenservice-copilots**: Support- of succesagents die productdocumentatie, FAQ's, CRM-notities en eerdere tickets gebruiken met grafiekbewuste opvraging en onderbouwde antwoorden.

- **Interne expert-copilots**: Beleids-, juridische of beveiligingsassistenten die redeneren over onderling verbonden regels, richtlijnen en historische beslissingen in plaats van geïsoleerde PDF's.

    Cognee is expliciet gepositioneerd als een persistent, accuraat geheugen voor AI-agents, en biedt een levend kennisgrafiek die achter je agent past en ad-hoc combinaties van vectoropslag en aangepaste grafiekcode vervangt.

2. **Data-silo's verenigen tot één geheugen**

    Dezelfde aanpak helpt je ook om een verenigde geheugenslaag te bouwen over verspreide bronnen:

- **Van silo's naar één grafiek**: Importeer gestructureerde (bijv. databases) en ongestructureerde gegevens (bijv. documenten, chats) in één grafiek ondersteund door embeddings, in plaats van aparte indices voor elk systeem.

- **Redeneren over bronnen met citaten**: Voer meerstapsredeneringen uit over alles—"combineer" logs, statistieken en documenten via de grafiek—en geef nog steeds onderbouwde antwoorden met herkomst.

- **Kenniscentra**: Voor domeinen zoals bankieren of onderwijs wordt Cognee al gebruikt om PDF's, interne systemen en app-gegevens te verenigen in één kennisgrafiek met vectors, zodat agents vragen kunnen beantwoorden met precieze, onderbouwde context.

## Volgende stappen

Je hebt de kerngeheugenlus geïmplementeerd. Hier zijn natuurlijke uitbreidingen die je zelf kunt proberen (zie [Cognee documentatie](https://docs.cognee.ai/) voor details):

1. **Experimenteer met tijdsbewustzijn**: Schakel tijds-cognify in om gebeurtenissen en tijdstempels uit tekst te extraheren.

2. **Introduceer op ontologie gebaseerde redenering**: Definieer een OWL-ontologie voor je domein. Gebruik Cognee's ontologie-ondersteuning zodat geëxtraheerde entiteiten en relaties in dat schema zijn verankerd, wat de grafiekkwaliteit en domeinspecifieke antwoorden verbetert.

3. **Voeg een feedbacklus toe**: Laat Cognee grafiekrandgewichten aanpassen op basis van echte gebruikersfeedback, zodat opvraging in de loop van de tijd verbetert in plaats van statisch te blijven.

4. **Afstemmen op personalisatie & sessiegedrag**: Gebruik gebruikers-ID's, tenants en datasets om elke persoon of team hun eigen weergave over de gedeelde geheugenslaag te geven.

5. **Schaal uit naar complexere agents**: Sluit Cognee aan op agent-frameworks om multi-agent systemen te bouwen die allemaal dezelfde geheugenslaag delen. *Microsoft Agent Framework x Cognee plugin komt binnenkort.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
